<a href="https://colab.research.google.com/github/soyoonjeong/PyTorch_Tutorial/blob/main/Optimizing_Model_Parameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터, 데이터로더, 신경망 모델

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root = 'data',
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform = ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size = 64)
test_dataloader = DataLoader(test_data, batch_size = 64)

class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork()

100%|██████████| 26421880/26421880 [00:02<00:00, 9873318.68it/s] 


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 199392.59it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3720858.03it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 4267248.42it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



# 최적화 단계

## 학습 단계

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    pred = model(X)
    loss = loss_fn(pred, y)

    # 역전파
    optimizer.zero_grad() # 모델 매개변수의 변화도 재설정, 기본적으로 변화도는 더해지기 때문에 중복 계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정
    loss.backward() # 예측 손실을 역전파
    optimizer.step() # 역전파 단계에서 수집된 변화도로 매개변수 조정

    if batch % 100 == 0:
      loss, current = loss.item(), (batch+1) *len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

## 검증/테스트 단계

In [6]:
def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error : \n Accuracy : {(100 * correct):>0.1f}%, Avg loss : {test_loss:>8f} \n")

## 실행

In [7]:
# 하이퍼파라미터
learning_rate = 1e-3 # 각 배치/에폭에서 모델의 매개변수를 조절하는 비율
batch_size = 64 #매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수
epochs = 5 # 데이터셋을 반복하는 횟수

# 손실함수 : 획득한 결과와 실제 값 사이의 틀린 정도를 측정하며 학습 중에 이 값을 최소화하려고 합니다.
loss_fn = nn.CrossEntropyLoss()

# 옵티마이저
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

for t in range(epochs):
  print(f"Epoch {t+1}\n----------------------------------")
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)
print('Done!')

Epoch 1
----------------------------------
loss: 2.300719 [   64/60000]
loss: 2.294544 [ 6464/60000]
loss: 2.271669 [12864/60000]
loss: 2.272706 [19264/60000]
loss: 2.251785 [25664/60000]
loss: 2.223552 [32064/60000]
loss: 2.238492 [38464/60000]
loss: 2.204646 [44864/60000]
loss: 2.205693 [51264/60000]
loss: 2.173919 [57664/60000]
Test Error : 
 Accuracy : 34.6%, Avg loss : 2.170108 

Epoch 2
----------------------------------
loss: 2.177046 [   64/60000]
loss: 2.169382 [ 6464/60000]
loss: 2.109457 [12864/60000]
loss: 2.133195 [19264/60000]
loss: 2.073181 [25664/60000]
loss: 2.022721 [32064/60000]
loss: 2.058679 [38464/60000]
loss: 1.984478 [44864/60000]
loss: 1.993052 [51264/60000]
loss: 1.913292 [57664/60000]
Test Error : 
 Accuracy : 55.4%, Avg loss : 1.917976 

Epoch 3
----------------------------------
loss: 1.948512 [   64/60000]
loss: 1.915752 [ 6464/60000]
loss: 1.801041 [12864/60000]
loss: 1.846506 [19264/60000]
loss: 1.722729 [25664/60000]
loss: 1.683141 [32064/60000]
loss: 1